# Setup

In [ ]:
# Installing required packages
!pip install pyspark
!pip install findspark
!pip install pyarrow==1.0.0
#!pip install pandas
#!pip install numpy==1.19.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=4990bfde6e0bca47bcafc69ca38e87f7860bf685a569ff5e35e6d334d6b7298d
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.3 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subproc

In [ ]:
import findspark
findspark.init()

In [ ]:
import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

## Spark Session

In [ ]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [ ]:
spark

## Load Part 1

In [ ]:
# Read the file using `read_csv` function in pandas
music = pd.read_csv('music_streaming.csv')

In [ ]:
# Preview a few records
music.head()

,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature,Genre
0,Bruno Mars,That's What I Like (feat. Gucci Mane),60.0,0.854,0.564,1.0,-4.964,1,0.0485,0.017100,NaN,0.0849,0.8990,134.071,234596.0,4,5
1,Boston,Hitch a Ride,54.0,0.382,0.814,3.0,-7.230,1,0.0406,0.001100,0.004010,0.1010,0.5690,116.454,251733.0,4,10
2,The Raincoats,No Side to Fall In,35.0,0.434,0.614,6.0,-8.334,1,0.0525,0.486000,0.000196,0.3940,0.7870,147.681,109667.0,4,6
3,Deno,Lingo (feat. J.I & Chunkz),66.0,0.853,0.597,10.0,-6.528,0,0.0555,0.021200,NaN,0.1220,0.5690,107.033,173968.0,4,5
4,Red Hot Chili Peppers,Nobody Weird Like Me - Remastered,53.0,0.167,0.975,2.0,-4.279,1,0.2160,0.000169,0.016100,0.1720,0.0918,199.060,229960.0,4,10


# Data Cleaning & Engineering

## Handling missing values

In [ ]:
# Handling Missing Values

# Deleting rows if missing percentage is very low would be simpler

missing_percentage = music['instrumentalness'].isnull().mean() * 100
print("Percentage of missing values in instrumentalness:", missing_percentage)

Percentage of missing values in instrumentalness: 23.11658181349488


In [ ]:
# Missing percentage is high, therefore we should use imputation: mean or median. We should find out if data is skewed first

music.describe()

,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature,Genre
count,15123.000000,15517.000000,15517.000000,13774.000000,15517.000000,15517.000000,15517.000000,15517.000000,11930.000000,15517.000000,15517.000000,15517.000000,1.551700e+04,15517.000000,15517.000000
mean,45.582358,0.541469,0.668711,5.964281,-7.856428,0.652639,0.080936,0.238502,0.177348,0.196837,0.487344,122.833548,1.992960e+05,3.925114,6.564155
std,17.400843,0.166838,0.235402,3.196333,4.018688,0.476147,0.084977,0.309605,0.302935,0.160212,0.240228,29.686429,1.148326e+05,0.360904,3.258829
min,1.000000,0.059600,0.001210,1.000000,-39.952000,0.000000,0.022500,0.000000,0.000001,0.011900,0.021500,30.557000,5.016500e-01,1.000000,0.000000
25%,34.000000,0.430000,0.517000,3.000000,-9.483000,0.000000,0.035100,0.003380,0.000097,0.097400,0.298000,99.729000,1.632400e+05,4.000000,5.000000
50%,45.000000,0.543000,0.708000,6.000000,-6.971000,1.000000,0.048300,0.068100,0.004215,0.129000,0.484000,120.065000,2.084270e+05,4.000000,8.000000
75%,57.000000,0.656000,0.866000,9.000000,-5.150000,1.000000,0.084800,0.410000,0.203000,0.260000,0.674000,142.378000,2.522530e+05,4.000000,10.000000
max,100.000000,0.989000,1.000000,11.000000,1.355000,1.000000,0.955000,0.996000,0.996000,1.000000,0.986000,217.416000,1.477187e+06,5.000000,10.000000


In [ ]:
#Statistics show that the "instrumentalness" column has a positively skewed distribution with a wide range of values,
#including some extreme values (ex: 0.000001). In this case, using median imputation might be a more suitable option than mean imputation
#because median imputation is more robust to outliers and can better represent the central tendency of a skewed distribution.

median_instrumentalness = music['instrumentalness'].median()
music['instrumentalness'] = music['instrumentalness'].fillna(median_instrumentalness)

In [ ]:
# Key column also has multiple missing values it seems.
# From the statistic, distribution of the "key" column is symmetric, with a mean value close to the median. The values range from 1 to 11
# Instead of mean, we should likely use mode insteda. this is because "key" values are discrete and represent musical keys

mode_key = music['key'].mode()[0]
music['key'] = music['key'].fillna(mode_key)

In [ ]:
# Let's see if any other columns suffer from missing data

# Check for missing values in each column
missing_values = music.isnull().sum()

# Display columns with missing values
columns_with_missing_values = missing_values[missing_values > 0]
print("Columns with missing values:\n", columns_with_missing_values)

Columns with missing values:
 Popularity    394
dtype: int64


In [ ]:
# Popularity column is left.
# From .describe we can tell popularit has a symmetric distribution (mean value close to the median among other indicators)
# We should impute using mean this time

mean_popularity = music['Popularity'].mean()
music['Popularity'] = music['Popularity'].fillna(mean_popularity)

In [ ]:
# Dealing with duplicates


# How many duplicate rows are there? *Spoiler* None
duplicate_rows = music.duplicated()
num_duplicates = duplicate_rows.sum()
print("Number of duplicate rows:", num_duplicates)

Number of duplicate rows: 0


##Data type conversions

In [ ]:
# Get the column types
column_types = music.dtypes

# Display the column types
print("Column Types:\n", column_types)

Column Types:
 Artist Name            object
Track Name             object
Popularity            float64
danceability          float64
energy                float64
key                   float64
loudness              float64
mode                    int64
speechiness           float64
acousticness          float64
instrumentalness      float64
liveness              float64
valence               float64
tempo                 float64
duration_in min/ms    float64
time_signature          int64
Genre                   int64
dtype: object


In [ ]:
# Data types seem fine.

## Data Integrity

There seems to be no logical outliers in the dataset, however

In [ ]:
# Data looks fine as it is, however one minor issue is that popularity column goes from 1 to 100, whereas similar columns like danceability,
#energy, speechiness, acousticness, instrumentalness, liveness and valence go from 0 to 1.

#It is a non-issue, however for the sake of uniformity, I'd like all the similar columns to have the same ranges.
# My preferred range is from 0 to 100.

from sklearn.preprocessing import MinMaxScaler

# Define the columns to be scaled
columns_to_scale = ['danceability', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence']

# Create an instance of MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 100))

# Scale the selected columns
music[columns_to_scale] = scaler.fit_transform(music[columns_to_scale])

In [ ]:
music.head()

,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature,Genre
0,Bruno Mars,That's What I Like (feat. Gucci Mane),60.0,85.474500,56.347180,1.0,-4.964,1,2.788204,1.716867,0.423093,7.387916,90.979782,134.071,234596.0,4,5
1,Boston,Hitch a Ride,54.0,34.689047,81.377467,3.0,-7.230,1,1.941019,0.110442,0.402510,9.017306,56.765163,116.454,251733.0,4,10
2,The Raincoats,No Side to Fall In,35.0,40.284054,61.353237,6.0,-8.334,1,3.217158,48.795181,0.019578,38.670175,79.367548,147.681,109667.0,4,6
3,Deno,Lingo (feat. J.I & Chunkz),66.0,85.366903,59.651178,10.0,-6.528,0,3.538874,2.128514,0.423093,11.142597,56.765163,107.033,173968.0,4,5
4,Red Hot Chili Peppers,Nobody Weird Like Me - Remastered,53.0,11.555842,97.496971,2.0,-4.279,1,20.750670,0.016968,1.616367,16.202813,7.288751,199.060,229960.0,4,10


## Duration column

In [ ]:
# There are some rows with minute values, and others with ms values. I want them all to be the same unit for consistency later on
# Logically the value should be in minutes.


# reame the column to more appropriate name
music.rename(columns={'duration_in min/ms': 'duration'}, inplace=True)

# Convert values greater than 25 to minutes (because that's the maximum value according to .describe())
music.loc[music['duration'] > 25, 'duration'] /= 60000

music.head()

,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration,time_signature,Genre
0,Bruno Mars,That's What I Like (feat. Gucci Mane),60.0,85.474500,56.347180,1.0,-4.964,1,2.788204,1.716867,0.423093,7.387916,90.979782,134.071,3.909933,4,5
1,Boston,Hitch a Ride,54.0,34.689047,81.377467,3.0,-7.230,1,1.941019,0.110442,0.402510,9.017306,56.765163,116.454,4.195550,4,10
2,The Raincoats,No Side to Fall In,35.0,40.284054,61.353237,6.0,-8.334,1,3.217158,48.795181,0.019578,38.670175,79.367548,147.681,1.827783,4,6
3,Deno,Lingo (feat. J.I & Chunkz),66.0,85.366903,59.651178,10.0,-6.528,0,3.538874,2.128514,0.423093,11.142597,56.765163,107.033,2.899467,4,5
4,Red Hot Chili Peppers,Nobody Weird Like Me - Remastered,53.0,11.555842,97.496971,2.0,-4.279,1,20.750670,0.016968,1.616367,16.202813,7.288751,199.060,3.832667,4,10


In [ ]:
# making sure that the changes were good and that there are no sneaking ms values
print(music['duration'].head(25))

0     3.909933
1     4.195550
2     1.827783
3     2.899467
4     3.832667
5     3.468883
6     5.489783
7     3.105783
8     3.964450
9     4.330450
10    3.715783
11    4.440250
12    2.561550
13    2.446667
14    2.571550
15    2.628217
16    4.716883
17    2.297867
18    2.300000
19    5.164450
20    2.420283
21    4.015633
22    4.289333
23    3.404667
24    3.286000
Name: duration, dtype: float64


In [ ]:
# On an unrelated note, I have to rename column Artist Name to Artist because it causes confusion in SQL queries later on.

music.rename(columns={'Artist Name': 'Artist'}, inplace=True)
music.rename(columns={'Track Name': 'Track'}, inplace=True)

In [ ]:
music.describe()

,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration,time_signature,Genre
count,15517.000000,15517.000000,15517.000000,15517.000000,15517.000000,15517.000000,15517.000000,15517.000000,15517.000000,15517.000000,15517.000000,15517.000000,15517.000000,15517.000000,15517.000000
mean,45.582358,51.847283,66.830926,6.080621,-7.856428,0.652639,6.266563,23.945968,13.787592,18.716461,48.299007,122.833548,3.934287,3.925114,6.564155
std,17.178491,17.951153,23.568726,3.029163,4.018688,0.476147,9.112804,31.084874,27.657306,16.214154,24.906979,29.686429,1.436557,0.360904,3.258829
min,1.000000,0.000000,0.000000,1.000000,-39.952000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,30.557000,0.000491,1.000000,0.000000
25%,34.000000,39.853669,51.641486,4.000000,-9.483000,0.000000,1.351206,0.339357,0.030221,8.652970,28.667703,99.729000,3.115583,4.000000,5.000000
50%,45.582358,52.012051,70.764625,7.000000,-6.971000,1.000000,2.766756,6.837349,0.423093,11.851027,47.952307,120.065000,3.666217,4.000000,8.000000
75%,57.000000,64.170433,86.583766,8.000000,-5.150000,1.000000,6.680965,41.164659,6.365368,25.108795,67.651633,142.378000,4.390000,4.000000,10.000000
max,100.000000,100.000000,100.000000,11.000000,1.355000,1.000000,100.000000,100.000000,100.000000,100.000000,100.000000,217.416000,24.619783,5.000000,10.000000


In [ ]:
music.to_csv('updated_music_streaming.csv')

# Loading Part 2 (Loading Data Into Spark DataFrame)

In [ ]:
sdf = spark.createDataFrame(music)

In [ ]:
sdf.printSchema()

root
 |-- Artist: string (nullable = true)
 |-- Track: string (nullable = true)
 |-- Popularity: double (nullable = true)
 |-- danceability: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- key: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: long (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- duration: double (nullable = true)
 |-- time_signature: long (nullable = true)
 |-- Genre: long (nullable = true)



In [ ]:
sdf.createTempView("songs")

# Running SQL queries and aggregating data

## b) Remove any songs that exceed 5 minutes


In [ ]:
# since the query says 'exceed', then we'll use the less than or equal symbol instead of less than
spark.sql("SELECT * FROM songs where duration <= 5").show(5)

+--------------------+--------------------+----------+------------------+------------------+----+--------+----+------------------+--------------------+--------------------+------------------+-----------------+-------+------------------+--------------+-----+
|              Artist|               Track|Popularity|      danceability|            energy| key|loudness|mode|       speechiness|        acousticness|    instrumentalness|          liveness|          valence|  tempo|          duration|time_signature|Genre|
+--------------------+--------------------+----------+------------------+------------------+----+--------+----+------------------+--------------------+--------------------+------------------+-----------------+-------+------------------+--------------+-----+
|          Bruno Mars|That's What I Lik...|      60.0|  85.4744996772111| 56.34718008790636| 1.0|  -4.964|   1|2.7882037533512065|   1.716867469879518|  0.4230927942698738| 7.387916202813481|90.97978227060652|134.071|3.9099333

##c) Display songs by J. Cole, Novo Amor and Anson Seabra

In [ ]:
spark.sql("SELECT * FROM songs WHERE Artist IN ('J. Cole', 'Novo Amor', 'Anson Seabra')").show(10)

+------------+--------------------+----------+-----------------+-----------------+----+--------+----+------------------+------------------+------------------+------------------+------------------+-------+------------------+--------------+-----+
|      Artist|               Track|Popularity|     danceability|           energy| key|loudness|mode|       speechiness|      acousticness|  instrumentalness|          liveness|           valence|  tempo|          duration|time_signature|Genre|
+------------+--------------------+----------+-----------------+-----------------+----+--------+----+------------------+------------------+------------------+------------------+------------------+-------+------------------+--------------+-----+
|     J. Cole|p u n c h i n ‚Äò...|      81.0|76.32881428878846|73.96850188728361|11.0|  -6.579|   0| 38.12332439678284|19.779116465863453|0.4230927942698738|16.101609148871574| 69.51788491446344| 94.104|1.8763166666666666|             4|    5|
|     J. Cole|p r i 

## d) How many songs are included in every category?


In [ ]:
spark.sql("SELECT Genre, COUNT(*) AS SongCount FROM songs GROUP BY Genre ORDER BY Genre ASC").show()

+-----+---------+
|Genre|SongCount|
+-----+---------+
|    0|      586|
|    1|     1268|
|    2|     1182|
|    3|      371|
|    4|      376|
|    5|     1210|
|    6|     2263|
|    7|      465|
|    8|     1704|
|    9|     1828|
|   10|     4264|
+-----+---------+



##e) Which artists dominated the charts?


In [ ]:
# Domination of the charts by an artist is determined by how ofen they appear, as well as how popular their songs are as well
# number of appearances is not enough because they could appear many times on the list but have mediocre songs.

# What's interesting is that from the results, Britney Spears has had more hit songs (greater total popularity) than the Backstreet Boys
# but they had 14 more songs than her in the dataset.

spark.sql("SELECT Artist, SUM(Popularity) AS TotalPopularity, COUNT(*) AS NumberOfSongs FROM songs GROUP BY Artist ORDER BY TotalPopularity DESC").show(5)

+------------------+------------------+-------------+
|            Artist|   TotalPopularity|NumberOfSongs|
+------------------+------------------+-------------+
|    Britney Spears| 2637.241221979766|           52|
|   Backstreet Boys|            2615.0|           66|
|The Rolling Stones|1838.3294319910071|           36|
|         Metallica|            1710.0|           27|
|                U2|            1648.0|           29|
+------------------+------------------+-------------+
only showing top 5 rows



##f) What songs would be considered for the “Billboard Top 10 Songs of the Year”? (mention their artists as well)


In [ ]:
# for a song to be top 10, the only important factor is number of listens throughout the year. I will assume that popularity is the factor
# closest to that.

spark.sql("SELECT Track, Artist, Popularity FROM songs ORDER BY Popularity DESC").show(10)

+--------------------+--------------------+----------+
|               Track|              Artist|Popularity|
+--------------------+--------------------+----------+
|MONTERO (Call Me ...|           Lil Nas X|     100.0|
|             Beggin'|            Måneskin|     100.0|
|            good 4 u|      Olivia Rodrigo|      99.0|
|Kiss Me More (fea...|            Doja Cat|      98.0|
|STAY (with Justin...|The Kid LAROI, Ju...|      97.0|
|Astronaut In The ...|         Masked Wolf|      97.0|
|          Bad Habits|          Ed Sheeran|      97.0|
|STAY (with Justin...|The Kid LAROI, Ju...|      97.0|
|              Butter|                 BTS|      96.0|
|             RAPSTAR|              Polo G|      96.0|
+--------------------+--------------------+----------+
only showing top 10 rows



## g) Recommend at least 5 songs that can be played at a party

In [ ]:
# I think the main parameters for a party song are: Liveness, Loudness, Energy and Danceability. Valence is not considered because party songs
# don't necessarily have to be happy. Duration is irrelevant. Speechiness is not a factor because there are EDM songs without words that
# work great in parties. The same reasoning goes for Instrumentalness. Tempo could be considered (as high tempo is synonymous with high energy
# which is great for parties) however there exist low tempo party songs. I think the most import factors are that the song is lively, loud,
# energetic and danceable.
# popularity could be a factor, however some unknown songs can be great party songs

# I wanted to keep those values as high as possible while still providing at least 5 results
# Loudness value kept at greater than 75% dataset
# We'll just  order by popularity

spark.sql("SELECT Track, Artist FROM songs WHERE Liveness > 85 AND Loudness > -10 AND Energy > 85 AND Danceability > 60  ORDER BY Popularity DESC").show()

+--------------------+----------+
|               Track|    Artist|
+--------------------+----------+
|I Like It, I Love It|Tim McGraw|
|         Got Me Good|     Ciara|
|     Good Times Roll|      GRiZ|
|Lovin' Is Everywhere| Engelwood|
|Stars - Live at S...|Simply Red|
|             Trigger| FEVER 333|
+--------------------+----------+

